In [2]:
!pip install keras_tuner -q

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import Hyperband

# Load your dataset or generate a sample dataset
# For example, let's use the built-in Breast Cancer dataset for binary classification
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

data = load_breast_cancer()
x_train, x_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.2, random_state=42)

# Define the model-building function for Keras Tuner
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.InputLayer(input_shape=(x_train.shape[1],)))

    # Tune the number of hidden layers and units
    for i in range(hp.Int('num_layers', 1, 4)):
        model.add(layers.Dense(units=hp.Int(f'units_{i}', 32, 256, 32),
                               activation='relu'))
        model.add(layers.Dropout(0.5))

    model.add(layers.Dense(1, activation='sigmoid'))

    # Tune the learning rate
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    return model

# Instantiate the tuner and perform hyperparameter search
tuner = Hyperband(build_model,
                  objective='val_accuracy',
                  max_epochs=10,
                  factor=3,
                  directory='keras_tuner_dir',
                  project_name='feedforward_nn')

tuner.search(x_train, y_train, epochs=10, validation_data=(x_test, y_test))

# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
best_model = tuner.hypermodel.build(best_hps)

# Train the best model
best_model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))


Reloading Tuner from keras_tuner_dir\feedforward_nn\tuner0.json
Epoch 1/10
15/15 [==============================] - 2s 30ms/step - loss: 50.4288 - accuracy: 0.5033 - val_loss: 9.0438 - val_accuracy: 0.6579
Epoch 2/10
15/15 [==============================] - 0s 11ms/step - loss: 10.3569 - accuracy: 0.6132 - val_loss: 0.2847 - val_accuracy: 0.9386
Epoch 3/10
15/15 [==============================] - 0s 11ms/step - loss: 0.6820 - accuracy: 0.7495 - val_loss: 0.3252 - val_accuracy: 0.9298
Epoch 4/10
15/15 [==============================] - 0s 10ms/step - loss: 0.5246 - accuracy: 0.7451 - val_loss: 0.2717 - val_accuracy: 0.8860
Epoch 5/10
15/15 [==============================] - 0s 10ms/step - loss: 0.5426 - accuracy: 0.7473 - val_loss: 0.2904 - val_accuracy: 0.9211
Epoch 6/10
15/15 [==============================] - 0s 10ms/step - loss: 0.4748 - accuracy: 0.7956 - val_loss: 0.2783 - val_accuracy: 0.9561
Epoch 7/10
15/15 [==============================] - 0s 10ms/step - loss: 0.4838 - accura